# DFT Manager
#### Austin Zadoks, Alejandro Strachan
---

In [1]:
%run launcher.py

In [2]:
install_libs()
show_projects()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…